# Module 2: Trident Installation

## Exercise 1: Installing Trident

In this exercise, you install NetApp Trident by using the manual operator method. 

You can also use Helm or the tridentctl method to install the Trident operator, but this exercise does not discuss
these other approaches.

**Objectives**

This exercise focuses on enabling you to do the following:

  - Download and set up the Trident operator
  - Deploy instances of Trident
  - Set up the tridentctl tool
  - Prepare worker nodes

**Exercise Equipment**

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |
| Kubernetes Worker 1     | kubwor1-1   | 192.168.0.62   | root                       | Netapp1!  |
| Kubernetes Worker 2     | kubwor1-2   | 192.168.0.63   | root                       | Netapp1!  |
| Kubernetes Worker 3     | kubwor1-3   | 192.168.0.64   | root                       | Netapp1!  |

**Prerequisites**

Before starting this exercise, you should take the following actions:

  -  Set up your Integrated Development Environment (IDE)
  -  Download the courseware GIT repository
  -  Configure your IDE to have access to your Kubernetes clusters
  -  Create svm0
  -  Configure svm0 to use the NFS v3 protocol


---
---


#### Task 1: Download and set Up the Trident operator

In this task, you verify that you can access the Kubernetes cluster, 

and you download and set up the Trident operator.


---

If desired, you can follow along with this exercise on the Trident operator deployment:

https://docs.netapp.com/us-en/trident/trident-get-started/kubernetes-deploy-operator.html#deploy-the-trident-operator-manually 

---

Verify that you have administrative access to the Kubernetes cluster:


In [ ]:
kubectl auth can-i '*' '*' --all-namespaces


---

In a future exercise, you implement Container Storage Interface (CSI) topologies. 

To support this effort, you apply different labels to each worker node. 

These labels should be present on the nodes in the cluster before you install Trident. 

The labels enable Trident to be topology-aware.


---

Label Worker 1 as Zone 1 and a region (for convenience, see [exercise2Task1-1.txt](./exercise2Task1-1.txt)):



In [ ]:
kubectl label node kubwor1-1 topology.kubernetes.io/region=trident topology.kubernetes.io/zone=zone1


---

Label Worker 2 as Zone 2 and a region (for convenience, see [exercise2Task1-2.txt](./exercise2Task1-2.txt)):



In [ ]:
kubectl label node kubwor1-2 topology.kubernetes.io/region=trident topology.kubernetes.io/zone=zone2

---

Label Worker 3 as Zone 3 and a region (for convenience, see [exercise2Task1-3.txt](./exercise2Task1-3.txt)):


In [ ]:
kubectl label node kubwor1-3 topology.kubernetes.io/region=trident topology.kubernetes.io/zone=zone3

---

Use a web browser to navigate to https://github.com/Netapp/trident/releases.

---

Identify the latest version of Trident at the top of the page

---

If desired, you can download a newer version. 

However, this exercise is not tested with any version other than 24.10.0. 

If you want to work with this version of the exercise, you can find the tar.gz file in the Exercise 2 folder in your class files.


---

Verify that you are in the `./Exercise 2`

In [ ]:
pwd

Unzip the Trident file:



In [ ]:
tar -xf trident-installer-24.10.0.tar.gz


A new subfolder, called trident-installer, should appear under the Exercise 2 folder.


---

cd to  Exercise 2 > trident-installer folder.

NOTE: This path serves as the relative path for all other paths in this task and the next task.


In [ ]:
cd trident-installer
pwd

---

Investigate the deploy/crds subfolder.


In [ ]:
ls -l deploy/crds

---

The crds subfolder contains several custom resource definition (CRD) YAML files.

Notice that three custom resource definitions (CRD) files with crd in the filenames

and six of these custom resources files with cr in the filenames.


---

Create the CRD definitions by using the `trident.netapp.io_tridentorchestrators_crd_post1.16.yaml` file:


In [ ]:
kubectl create -f deploy/crds/trident.netapp.io_tridentorchestrators_crd_post1.16.yaml


---

In the Kubernetes Extension of your IDE, you should see the
`tridentorchestrators` CRD.

Expand **Clusters**> **source-admin@source**> **Custom Resources** to view in the Kubernetes Extension.

If you see an error under the tridentorchestrators CRD, click the Refresh button to
make it disappear.


---

Create the trident namespace:


In [ ]:
kubectl create -f deploy/namespace.yaml


---

Copy and rename the resulting file for the aggregated YAML “kustomized” file for Kubernetes
version 1.25 or later (for convenience, see [exercise2Task1-4.txt](./exercise2Task1-4.txt)):


In [ ]:

cp deploy/kustomization_post_1_25.yaml deploy/kustomization.yaml


NOTE: 

This kustomization.yaml file runs the 

- serviceaccount.yaml, 
- clusterrolebinding.yaml, and the 
- operator.yaml 

files.


---

Create a YAML bundle that you can run (for convenience, see [exercise2Task1-5.txt](./exercise2Task1-5.txt)):



In [ ]:
kubectl kustomize deploy/ > deploy/bundle_post_1_25.yaml

---

Install the operator (for convenience, see [exercise2Task1-6.txt](./exercise2Task1-6.txt)):



In [ ]:
kubectl create -f deploy/bundle_post_1_25.yaml

---

Verify that you created all the objects:

`kubectl -n trident get all`

Sample output:

```terminal
NAME READY STATUS RESTARTS AGE
pod/trident-operator-5c94fc5556-nlsnl 1/1 Running 0 2m7s
NAME READY UP-TO-DATE AVAILABLE AGE
deployment.apps/trident-operator 1/1 1 1 2m7s
NAME DESIRED CURRENT READY AGE
replicaset.apps/trident-operator-5c94fc5556 1 1 1 2m7s
```

In [ ]:
kubectl -n trident get all

---

A Kubernetes cluster should contain only one instance of the operator. 

You must not create multiple deployments of the Trident operator.


---
---

#### Task 2: Deploy instances of Trident

In this task, you use the operator to deploy Trident. 

This action requires you to create a `TridentOrchestrator` custom resource (CR). 

The Trident installer includes example definitions for creating the `TridentOrchestrator` CR. 

This CR starts an installation in the trident namespace.

The relative path for this exercise is the Exercise 2 > trident-installer folder.



---

Review the [deploy/crds/tridentorchestrator_cr.yaml](./trident-installer/deploy/crds/tridentorchestrator_cr.yaml) file.


---

Create an instance of the `TridentOrchestrator` CR:


In [ ]:
kubectl create -f deploy/crds/tridentorchestrator_cr.yaml


---

In the Kubernetes Extension of your IDE, you should see the trident instance
under the `tridentorchestrators` CRD.


---

Review the details by double-clicking the trident entry in the hierarchy, or use the following
command:


In [ ]:

kubectl -n trident describe torc trident


---

Answer the following question:

In the events section, what is the last event type and reason?


---

Verify that you created all objects:


Sample output:

```terminal
NAME READY STATUS RESTARTS AGE
pod/trident-controller-74698976f5-5d2tz 6/6 Running 0 3m16s
pod/trident-node-linux-8t2tp 2/2 Running 2 (60s ago) 3m16s
pod/trident-node-linux-dnbm6 2/2 Running 2 (77s ago) 3m16s
pod/trident-node-linux-k29t7 2/2 Running 2 (76s ago) 3m16s
pod/trident-node-linux-td97m 2/2 Running 2 (71s ago) 3m16s
pod/trident-operator-5c94fc5556-nlsnl 1/1 Running 0 13h
NAME TYPE CLUSTER-IP EXTERNAL-IP PORT(S)
service/trident-csi ClusterIP 10.102.83.55 <none> 34571/TCP,9220/TCP 3
NAME DESIRED CURRENT READY UP-TO-DATE AVAIL
daemonset.apps/trident-node-linux 4 4 4 4 4
NAME READY UP-TO-DATE AVAILABLE AGE
deployment.apps/trident-controller 1/1 1 1 3m16s
deployment.apps/trident-operator 1/1 1 1 13h
NAME DESIRED CURRENT READY AGE
replicaset.apps/trident-controller-74698976f5 1 1 1 3m16s
replicaset.apps/trident-operator-5c94fc5556 1 1 1 13h
```

In [ ]:
kubectl -n trident get all


---

The DaemonSet `trident-node-linux` creates the four `trident-node-linux` pods. 

One `trident-node-linux` pod is installed on each node (including the control-plane master node). 

The `trident-controller` deployment creates the `trident-controller` pod, which runs on one of the worker nodes.


---

Stop the deployed Trident pods by deleting the TridentOrchestrator CR:


In [ ]:
kubectl -n trident delete torc trident


---

Verify that every pod with node or controller in its name is deleted and that only the
Trident operator is running:


In [ ]:
kubectl -n trident get pods -o wide


The TridentOrchestrator CR enables you to customize the Trident operator.

See the following URL for more details: 

https://docs.netapp.com/us-en/trident/trident-get-started/kubernetes-customize-deploy.html.

The “crds” subfolder contains several examples of modifications.


---

Try to run Trident only on Worker 1 and Worker 3.

---

Create labels on two worker nodes:


In [ ]:
kubectl label node kubwor1-1 storage=trident
kubectl label node kubwor1-3 storage=trident


---

Edit the [deploy/crds/tridentorchestrator_cr.yaml](./trident-installer/deploy/crds/tridentorchestrator_cr.yaml) file to add an appropriate toleration:

(see https://docs.netapp.com/us-en/trident/trident-get-started/kubernetes-customize-deploy.html)

 -  Definition: **nodePluginNodeSelector** 
 -  Key: storage
 -  Value: trident

<details> <summary>Solution  </summary>

You can find the solution for this step in the [exercise2Task2-nodeselector.yaml](./Solutions/exercise2Task2-nodeselector.yaml) file.


---

Create an instance of the TridentOrchestrator CR:


In [ ]:
kubectl create -f ../Solutions/exercise2Task2-nodeselector.yaml

---

After a few minutes, verify that the trident controller and node pods are only running on
kubwor1-1 and kubwor1-3:


In [ ]:
kubectl -n trident get pods -o wide


In [ ]:
kubectl -n trident get pods -o wide


In [ ]:
kubectl -n trident get pods -o wide


---

Add the label to Worker 2:


In [ ]:
kubectl label node kubwor1-2 storage=trident


---

Verify which nodes trident is running on:


In [ ]:
kubectl -n trident get pods -o wide


---


CHALLENGE STEP: 

You can update an existing instance of the trident CR by using a patch command. 

For example, if you want to turn off debug logs, use the following command:



In [ ]:
kubectl -n trident patch torc trident --type=json -p '[{"op": "replace", "path": "/spec/debug", "value": "false"}]'


---

CHALLENGE STEP: 

Verify that the debug logs are off.


---

In [ ]:
kubectl -n trident describe torc trident 


Turn on debug logs

Verify that the debug logs are on.


In [ ]:
kubectl -n trident patch torc trident --type=json -p '[{"op": "replace", "path": "/spec/debug", "value": "true"}]'


In [ ]:
kubectl -n trident describe torc trident 


---
---

#### Task 3: Set up the tridentctl Tool

The tridentctl tool was installed when you unzipped the trident-installer file.


---

From the Exercise 2 folder, execute [exercise2Task3.sh](./exercise2Task3.sh) file from a terminal:


In [ ]:
cd ~
target_dir=$(find . -type d -name 'STRSW-ILT-UATWK*' 2>/dev/null | head -n 1)
if [ -n "$target_dir" ]; then
    cd "$target_dir"
else
    echo "Directory not found"
fi
cd .'/Exercise 2'

./exercise2Task3.sh


---

Review the tridentctl subcommands:

tridentctl

Sample output:

```terminal
A CLI tool for managing the NetApp Trident external storage provisioner for Kubernetes
Usage:
tridentctl [command]
Available Commands:
completion Generate the autocompletion script for the specified shell
create Add a resource to Trident
delete Remove one or more resources from Trident
get Get one or more resources from Trident
help Help about any command
images Print a table of the container images Trident needs
import Import an existing resource to Trident
install Install Trident
logs Print the logs from Trident
send Send a resource from Trident
uninstall Uninstall Trident
update Modify a resource in Trident
version Print the version of Trident
Flags:
-d, --debug Set the log level to debug
-h, --help help for tridentctl
-k, --kubeconfig string Kubernetes config path
--log-level string Log level (trace, debug, warn, info, error, fatal (default "info")
-n, --namespace string Namespace of Trident deployment
-o, --output string Output format. One of json|yaml|name|wide|ps (default)
-s, --server string Address/port of Trident REST interface (127.0.0.1 or [::1] only)
Use "tridentctl [command] --help" for more information about a command.


In [ ]:
tridentctl


---

Verify which version of Trident is installed:
tridentctl -n trident version

```terminal
Sample output:
+----------------+----------------+
| SERVER VERSION | CLIENT VERSION |
+----------------+----------------+
| 24.10.0 | 24.10.0 |
+----------------+----------------+
```


In [ ]:
tridentctl -n trident version


---

See the images that are required for Trident to function, per the Kubernetes version:

`tridentctl -n trident images`

```terminal
Sample output:
…
+--------------------+---------------------------------------------------------------+
| v1.31.0 | netapp/trident:24.10.0 |
| | docker.io/netapp/trident-autosupport:24.10 |
| | registry.k8s.io/sig-storage/csi-provisioner:v5.1.0 |
| | registry.k8s.io/sig-storage/csi-attacher:v4.7.0 |
| | registry.k8s.io/sig-storage/csi-resizer:v1.12.0 |
| | registry.k8s.io/sig-storage/csi-snapshotter:v8.1.0 |
| | registry.k8s.io/sig-storage/csi-node-driver-registrar:v2.12.0 |
| | netapp/trident-operator:24.10.0 (optional) |
+--------------------+---------------------------------------------------------------+


In [ ]:

tridentctl -n trident images

---
---

#### Task 4: Prepare worker nodes

In this task, you verify that the worker nodes can use the volumes that Trident provides.


---

Open a Secure Shell (SSH) session to Worker 1:
ssh root@192.168.0.62

You can also use the code cells to execute the code

---

Verify that nfs-common is installed:


In [ ]:
ssh root@kubwor1-1 apt list --installed | grep nfs-common


---

Verify that open-iscsi, lsscsi, and scsitools are installed:


In [ ]:
ssh root@kubwor1-1  apt list --installed | grep scsi



---

Verify that sg3-utils is installed:


In [ ]:
ssh root@kubwor1-1  apt list --installed | grep sg3


---

Verify that multipath-tools is installed:


In [ ]:
ssh root@kubwor1-1  apt list --installed | grep multipath


---

Verify that /etc/multipath.conf has the following values :
defaults {
user_friendly_names yes
find_multipaths no
}


In [ ]:
ssh root@kubwor1-1 cat /etc/multipath.conf

---

Enable multipathing:


In [ ]:
ssh root@kubwor1-1  systemctl enable --now iscsid multipathd


NOTE: If you see an error, please ignore it.


In [ ]:
ssh root@kubwor1-1  "service iscsid restart && service multipathd restart"


---

Verify that multipath-tools and iscsid and are enabled and running:


In [ ]:
ssh root@kubwor1-1  systemctl status multipathd iscsid


---

Verify your initiator node name:


In [ ]:
ssh root@kubwor1-1 cat /etc/iscsi/initiatorname.iscsi


---

Verify that nvme-cli is installed:


In [ ]:
ssh root@kubwor1-1  apt list --installed | grep nvme


---

Scan the NVMe bus:


In [ ]:
ssh root@kubwor1-1  modprobe nvme-tcp


---

Verify your NVMe Qualified Name (NQN):


In [ ]:
ssh root@kubwor1-1 cat /etc/nvme/hostnqn



---

Implement these steps across all nodes in the source and destination clusters:

[./exercise2Task4.sh](./exercise2Task4.sh)


In [ ]:

./exercise2Task4.sh


End of exercise